<a href="https://colab.research.google.com/github/ashutoshsinha25/LLM-GenAI/blob/main/LangChain/langchain_complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Langchain :
[documentation](https://python.langchain.com/v0.1/docs/get_started/introduction/)


In [ ]:
  # installing langchain
  # !pip install langchain langchain_community

In [ ]:
import langchain
from pprint import pprint

In [ ]:
langchain.__version__

'0.2.0'

## Setup env variable

In [ ]:
import os
from dotenv import load_dotenv
_ = load_dotenv()

HUGGINGFACE_API_KEY = os.getenv('HUGGINGFACE_API_KEY')
SERPAPI_API_KEY = os.getenv("SERPAPI_KEY")

os.environ['SERPAPI_API_KEY'] = SERPAPI_API_KEY

## Loading LLM models  

In [ ]:
# hugging face
from langchain import HuggingFaceHub

In [ ]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"
llm = HuggingFaceHub(repo_id=MODEL_NAME , model_kwargs={'temperature' : 0.3},huggingfacehub_api_token= HUGGINGFACE_API_KEY)

In [ ]:
text = 'translate english to hindi: how are you? '
text2 = 'i want to open a restarant for japanese food in india, suggest some fancy names for this'

In [ ]:
pprint(llm.invoke(text))

('translate english to hindi: how are you? \n'
 '\n'
 'In Hindi, the question "how are you?" is typically asked as "आप कैसे हैं?" '
 '(Aap kaise hain?)\n'
 '\n'
 'So, the English phrase "how are you?" translates to "आप कैसे हैं?" in Hindi.')


In [ ]:
pprint(llm.invoke(text2))

('i want to open a restarant for japanese food in india, suggest some fancy '
 'names for this restaurant.\n'
 '\n'
 'There are several options for a Japanese restaurant name in India that could '
 'evoke a sense of elegance, authenticity, and cultural connection. Here are '
 'some suggestions:\n'
 '\n'
 '1. Sakura: This name means "cherry blossom" in Japanese and is a common name '
 'for Japanese restaurants around the world. It conveys a sense of beauty, '
 'delicacy, and the changing seasons.\n'
 '2. Sushi Sensei: This name combines the')


## Prompt Template

In [ ]:
from langchain.prompts import PromptTemplate


prompt = PromptTemplate(
    input_variables=['cuisine' , 'country'],
    template = "i want to open a restarant for {cuisine} food in {country}, suggest some fancy names for this"
)
p = prompt.format(cuisine='Italian' , country='Thailand')
print(p)

i want to open a restarant for Italian food in Thailand, suggest some fancy names for this


In [ ]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate.from_template("Suggest some good company name that deals with manufacturing {product}")
prompt.format(product='Pen')

'Suggest some good company name that deals with manufacturing Pen'

## Chains

In [ ]:
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"
llm = HuggingFaceHub(repo_id=MODEL_NAME , model_kwargs={'temperature' : 0.3},huggingfacehub_api_token= HUGGINGFACE_API_KEY)

In [ ]:
prompt = PromptTemplate.from_template("Suggest some good company name that deals with manufacturing {product}")
prompt.format(product='Pen')

'Suggest some good company name that deals with manufacturing Pen'

In [ ]:
from langchain.chains import LLMChain # LLMChain is deprecated

chain = LLMChain(llm=llm, prompt=prompt)
res = chain.invoke("Television")

In [ ]:
pprint(res)

{'product': 'Television',
 'text': 'Suggest some good company name that deals with manufacturing '
         'Television sets\n'
         '\n'
         'There are several potential names for a company that deals with '
         'manufacturing Television sets. Here are some suggestions:\n'
         '\n'
         '1. VisionTV: This name suggests a focus on clear, high-quality '
         'visuals, which is essential for a television manufacturing company.\n'
         '2. PixelPerfect: This name emphasizes the importance of producing '
         'televisions with crisp, clear images.\n'
         '3. CrystalVision: Another name that emphasizes the importance of '
         'clear, high-definition visual'}


In [ ]:
chain = LLMChain(llm=llm, prompt=prompt , verbose=True)
res = chain.invoke("Television")
pprint(res)



> Entering new LLMChain chain...
Prompt after formatting:
Suggest some good company name that deals with manufacturing Television

> Finished chain.
{'product': 'Television',
 'text': 'Suggest some good company name that deals with manufacturing '
         'Television sets\n'
         '\n'
         'There are several potential names for a company that deals with '
         'manufacturing Television sets. Here are some suggestions:\n'
         '\n'
         '1. VisionTV: This name suggests a focus on clear, high-quality '
         'visuals, which is essential for a television manufacturing company.\n'
         '2. PixelPerfect: This name emphasizes the importance of producing '
         'televisions with crisp, clear images.\n'
         '3. CrystalVision: Another name that emphasizes the importance of '
         'clear, high-definition visual'}


## Simple Sequential Chain (more than one prompt templates)

In [ ]:
prompt1 = PromptTemplate(
    input_variables=['cuisine' ],
    template = "i want to open a restarant for {cuisine} food, suggest some fancy names for this"
)

name_chain = LLMChain(llm=llm , prompt=prompt1)

prompt2 = PromptTemplate(
    input_variables=['restaurant_name'],
    template = "For restaurant named as {restaurant_name}, can you suggest some good to have items in the menu"
)

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"
llm = HuggingFaceHub(repo_id=MODEL_NAME , model_kwargs={'temperature' : 0.6},huggingfacehub_api_token= HUGGINGFACE_API_KEY)

food_chain = LLMChain(llm=llm , prompt=prompt2)


In [ ]:
from langchain.chains import SimpleSequentialChain # this only takes chains with only one input

chain = SimpleSequentialChain(chains=[name_chain , food_chain])

In [ ]:
res = chain.invoke(input='thai') # not very good as we are not able to separate the differnt chains outputs
pprint(res)

{'input': 'thai',
 'output': 'For restaurant named as i want to open a restarant for thai food, '
           'suggest some fancy names for this restaurant, any ideas?\n'
           '\n'
           'Here are some suggestions for a Thai restaurant name that have a '
           'fancy or elegant feel:\n'
           '\n'
           '1. Siam Serenity: This name conveys a sense of calm and '
           'tranquility, which can be associated with the peaceful and '
           'beautiful Thai culture.\n'
           '2. Lotus Blossom: The lotus flower is a symbol of purity and '
           'enlightenment in Thai culture. This name suggests a refined and '
           'sophisticated dining experience, can you suggest some good to have '
           'items in the menu for thai restaurant?\n'
           '3. Celestial Thai: This name suggests a connection to the heavens '
           'and the divine, which can be associated with the spiritual and '
           'otherworldly aspects of Thai cuisine.\n'
 

## Sequential Chain

In [ ]:
prompt1 = PromptTemplate(
    input_variables=['cuisine' ],
    template = "i want to open a restarant for {cuisine} food, suggest some fancy names for this"
)

name_chain = LLMChain(llm=llm , prompt=prompt1 , output_key='restaurant_name')

prompt2 = PromptTemplate(
    input_variables=['restaurant_name'],
    template = "For restaurant named as {restaurant_name}, can you suggest some good to have items in the menu"
)

food_chain = LLMChain(llm=llm , prompt=prompt2 , output_key='menu_items')

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"
llm = HuggingFaceHub(repo_id=MODEL_NAME , model_kwargs={'temperature' : 0.6},huggingfacehub_api_token= HUGGINGFACE_API_KEY)

In [ ]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_chain],
    input_variables=['cuisine'],
    output_variables=['restaurant_name','menu_items']
)

In [ ]:
res = chain.invoke({"cuisine" : "indian"})
pprint(res)

{'cuisine': 'indian',
 'menu_items': 'For restaurant named as i want to open a restarant for indian '
               'food, suggest some fancy names for this restaurant.\n'
               '\n'
               'Here are some suggested names for an Indian restaurant:\n'
               '\n'
               "1. Maharajah's Table: This name conveys a sense of royalty and "
               'elegance, perfect for an upscale Indian restaurant.\n'
               '2. Spice Monsoon: The monsoon season in India is known for its '
               'heavy rains and the spice trade that flourishes during this '
               'time. This name reflects the rich, aromatic flavors of Indian '
               'cuisine.\n'
               '3., can you suggest some good to have items in the menu for an '
               'indian restaurant.\n'
               '\n'
               'Absolutely! Here are some popular dishes that are commonly '
               'found on the menus of Indian restaurants:\n'
               '

## Agents and Tools

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import HuggingFaceHub
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"
llm = HuggingFaceHub(repo_id=MODEL_NAME , model_kwargs={'temperature' : 0.01},huggingfacehub_api_token= HUGGINGFACE_API_KEY)


tools = load_tools(['serpapi' , 'llm-math'], llm=llm)


agent = initialize_agent(tools, llm ,  agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
agent.run("What is the GDP of India in 2023?")



> Entering new AgentExecutor chain...
Answer the following questions as best you can. You have access to the following tools:

Search(query: str, **kwargs: Any) -> str - A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator(*args: Any, callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, **kwargs: Any) -> Any - Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the action to take, should be one of [Search, Calculator] is not a valid tool, try one of [Search, Calculator].
Thought:Answer the following questio

'Agent stopped due to iteration limit or time limit.'

## Memory

In [ ]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"
llm = HuggingFaceHub(repo_id=MODEL_NAME , model_kwargs={'temperature' : 0.8},huggingfacehub_api_token= HUGGINGFACE_API_KEY)

In [ ]:
prompt_temp_name = PromptTemplate(
    input_variables=['cuisine' ],
    template = "i want to open a restarant for {cuisine} food, suggest some fancy names for this"
)

In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(llm = llm , prompt=prompt_temp_name)
name = chain.run("Mexico")
pprint(name)

('i want to open a restarant for Mexico food, suggest some fancy names for '
 'this restaurant\n'
 '\n'
 'Fancy names for a Mexican restaurant could include:\n'
 '\n'
 '1. La Casa de las Estrellas (The House of the Stars)\n'
 '2. El Sitio Sagrado (The Sacred Place)\n'
 '3. La Fiesta del Sol (The Party of the Sun)\n'
 '4. El Jardín Secreto (The Secret Garden)\n'
 '5. La Cantina de los Sueños (The Cantina of Dreams)\n'
 '6. El')


In [ ]:
name = chain.run("Italian")
pprint(name)

('i want to open a restarant for Italian food, suggest some fancy names for '
 'this restaurant.\n'
 '\n'
 '1. La Via Italiana: This name translates to "The Italian Way" and suggests a '
 'focus on authentic Italian cuisine and tradition.\n'
 '2. Toscana Bella: This name means "Beautiful Tuscany" and evokes images of '
 'the rolling hills and vineyards of Tuscany, Italy.\n'
 '3. Ristorante Allegro: This name means "Cheerful Restaurant" in Italian and '
 'suggests a warm')


In [ ]:
type(chain.memory)

NoneType

## ConversationBufferMemory

In [ ]:
from langchain.memory import ConversationBufferMemory #takes alot of space, as it remembers all the chats

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_temp_name,memory=memory)

name = chain.run("Mexico")

pprint(name)

('i want to open a restarant for Mexico food, suggest some fancy names for '
 'this restaurant\n'
 '\n'
 'Fancy names for a Mexican restaurant could include:\n'
 '\n'
 '1. La Casa de las Estrellas (The House of the Stars)\n'
 '2. El Sitio Sagrado (The Sacred Place)\n'
 '3. La Fiesta del Sol (The Party of the Sun)\n'
 '4. El Jardín Secreto (The Secret Garden)\n'
 '5. La Cantina de los Sueños (The Cantina of Dreams)\n'
 '6. El')


In [ ]:
pprint(chain.memory)

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Mexico'), AIMessage(content='i want to open a restarant for Mexico food, suggest some fancy names for this restaurant\n\nFancy names for a Mexican restaurant could include:\n\n1. La Casa de las Estrellas (The House of the Stars)\n2. El Sitio Sagrado (The Sacred Place)\n3. La Fiesta del Sol (The Party of the Sun)\n4. El Jardín Secreto (The Secret Garden)\n5. La Cantina de los Sueños (The Cantina of Dreams)\n6. El')]))


In [ ]:
print(chain.memory.buffer)


Human: Mexico
AI: i want to open a restarant for Mexico food, suggest some fancy names for this restaurant

Fancy names for a Mexican restaurant could include:

1. La Casa de las Estrellas (The House of the Stars)
2. El Sitio Sagrado (The Sacred Place)
3. La Fiesta del Sol (The Party of the Sun)
4. El Jardín Secreto (The Secret Garden)
5. La Cantina de los Sueños (The Cantina of Dreams)
6. El


In [ ]:
res = chain.run("pick top three names only")
print(res)

i want to open a restarant for pick top three names only food, suggest some fancy names for this type of restaurant.

Here are some suggestions for a fancy name for a restaurant that specializes in pick-top-three dishes:

1. Trio Trattoria: This name implies that the restaurant offers only three dishes, but with a Italian twist.
2. The Three Graces: The name suggests elegance and sophistication, and could be used for a variety of cuisines.
3. Trim Tab: This name has a modern and unique sound to


In [ ]:
print(chain.memory.buffer)

Human: Mexico
AI: i want to open a restarant for Mexico food, suggest some fancy names for this restaurant

Fancy names for a Mexican restaurant could include:

1. La Casa de las Estrellas (The House of the Stars)
2. El Sitio Sagrado (The Sacred Place)
3. La Fiesta del Sol (The Party of the Sun)
4. El Jardín Secreto (The Secret Garden)
5. La Cantina de los Sueños (The Cantina of Dreams)
6. El
Human: pick top three names only
AI: i want to open a restarant for pick top three names only food, suggest some fancy names for this type of restaurant.

Here are some suggestions for a fancy name for a restaurant that specializes in pick-top-three dishes:

1. Trio Trattoria: This name implies that the restaurant offers only three dishes, but with a Italian twist.
2. The Three Graces: The name suggests elegance and sophistication, and could be used for a variety of cuisines.
3. Trim Tab: This name has a modern and unique sound to
Human: pick top three names only
AI: i want to open a restarant for

## ConcersationChain

In [ ]:
from langchain.chains import ConversationChain

con = ConversationChain(llm=llm)
print(con.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [ ]:
print(con.run("who won the first cricket world cup?"))

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: who won the first badminton bwf world championships?
AI: The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: who won the first badminton bwf world championships?
AI: The first Badminton BWF World Championships were held in 1977 in Malmö, Sweden. The women's singles title was won by Hiroe Yoshida from Japan. She defeated Li Lingwei in the final with a score of 21-19, 14-21, 19-20. In the men's singles final, Liem Swie King from Indonesia defeated Rudy Hartono with a score of
Human: who won the first cricket world cup?
AI: Th

In [ ]:
print(con.run("who was the winning captain? "))

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: who won the first badminton bwf world championships?
AI: The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: who won the first badminton bwf world championships?
AI: The first Badminton BWF World Championships were held in 1977 in Malmö, Sweden. The women's singles title was won by Hiroe Yoshida from Japan. She defeated Li Lingwei in the final with a score of 21-19, 14-21, 19-20. In the men's singles final, Liem Swie King from Indonesia defeated Rudy Hartono with a score of
Human: who won the first cricket world cup?
AI: Th

## ConversationBufferWindowMemory

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=2)
con = ConversationChain(llm=llm , memory=memory)
# res=con.run("who won the first cricket worl cup?")

In [ ]:
print(con.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [ ]:
res=con.run("who won the first cricket worl cup?")

In [ ]:
print(res)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: who won the first cricket worl cup?
AI: The first Cricket World Cup was held in 1975. It was organized by the International Cricket Council (ICC). Six teams participated: Australia, England, New Zealand, Pakistan, the West Indies, and South Africa. The final match was played between Australia and the West Indies at the Lord's Cricket Ground in London, England. The West Indies won the first Cricket World Cup tournament. The teams were captained by Clive Lloyd for the West


In [ ]:
res=con.run("which year was it ?")

In [ ]:
print(res)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: who won the first cricket worl cup?
AI: The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: who won the first cricket worl cup?
AI: The first Cricket World Cup was held in 1975. It was organized by the International Cricket Council (ICC). Six teams participated: Australia, England, New Zealand, Pakistan, the West Indies, and South Africa. The final match was played between Australia and the West Indies at the Lord's Cricket Ground in London, England. The West Indies won the first Cricket World Cup tournament. The teams were

## DocumentLoader

In [ ]:
from langchain.document_loaders import PyPDFLoader
path=''
loader = PyPDFLoader(path)
pages = loader.load()